In [1]:
import numpy as np 
import pandas as pd 
import pickle

from FDSSC import models, utils
from datasets.datasets import LocationChecker, FDSSCDataset, RandomRotation
from matplotlib import pyplot as plt
import torch
from torch.utils.data import DataLoader

In [ ]:
# process data to 

In [2]:
datadir = "data/stacked_images"
labeldir = "siteData/allData/"
checker = LocationChecker(datadir, size=(9, 9), rigorous=False)
trainLandsat8Data = FDSSCDataset(datadir, labeldir+"train.csv", lochecker=checker, transform=[RandomRotation()])
valLandsat8Data = FDSSCDataset(datadir, labeldir+"val.csv", lochecker=checker)
testLandsat8Data = FDSSCDataset(datadir, labeldir+"test.csv", lochecker=checker)
trainLoader = DataLoader(trainLandsat8Data, batch_size=8, shuffle=True, num_workers=4)
valLoader = DataLoader(valLandsat8Data, batch_size=16, num_workers=1)
testLoader = DataLoader(testLandsat8Data, batch_size=16, num_workers=1)
#id2idx = utils.get_id2idx(pd.read_csv(csv), "MAJOR_COMMODITY_CODE")
id2idx = pickle.load(open("data/id2idxMasked.p", "rb"))
idx2id = {value: key for key, value in id2idx.items()}

In [3]:
# setting training device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# initializing model and loss and optim
model = models.FerDSSC_model((1, 9, 9, 11), len(list(id2idx.keys())))
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.RMSprop(model.parameters(), lr=0.0003)

# applying weight initialization
model.apply(utils.init_weights)

#putting model on training device
model.to(device)


FerDSSC_model(
  (input_spec_conv): Conv3d(1, 24, kernel_size=(1, 1, 7), stride=(1, 1, 1), padding=(0, 0, 3))
  (spectral_conv1): Spectral_conv(
    (bn_prelu): Bn_prelu(
      (bn): BatchNorm3d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (prelu): PReLU(num_parameters=1)
    )
    (conv3d): Conv3d(24, 12, kernel_size=(1, 1, 7), stride=(1, 1, 1), padding=(0, 0, 3))
  )
  (spectral_conv2): Spectral_conv(
    (bn_prelu): Bn_prelu(
      (bn): BatchNorm3d(36, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (prelu): PReLU(num_parameters=1)
    )
    (conv3d): Conv3d(36, 12, kernel_size=(1, 1, 7), stride=(1, 1, 1), padding=(0, 0, 3))
  )
  (spectral_conv3): Spectral_conv(
    (bn_prelu): Bn_prelu(
      (bn): BatchNorm3d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (prelu): PReLU(num_parameters=1)
    )
    (conv3d): Conv3d(48, 12, kernel_size=(1, 1, 7), stride=(1, 1, 1), padding=(0, 0, 3))
  )
  (bn_prelu1): Bn

In [ ]:
print(trainLandsat8Data[0])

In [4]:
epochs = 5
frequency = 5
# setting up callbacks
checkpointFolder = "TrainingCheckpoints/3rdRun/"
cbs = [utils.validate_callback(model, valLoader, loss_fn, device),
        utils.saving_checkpoints_callback(checkpointFolder, model, optimizer, frequency, epochs)]

trainingLosses, CBouts = utils.train(model, trainLoader, epochs=epochs, loss_fn=loss_fn,
                            optimizer=optimizer, callbacks=cbs)


======== Epoch 1 / 5 ========
Training...
  Batch   311  of    312.    Loss: 2.85     Elapsed: 0:01:19.
  Average training loss: 1.81
  Training epoch took: 0:01:19
Running Validation...
  Accuracy: 58.17
  Validation took: 0:00:25

======== Epoch 2 / 5 ========
Training...
  Batch   311  of    312.    Loss: 2.37     Elapsed: 0:01:18.
  Average training loss: 1.79
  Training epoch took: 0:01:19
Running Validation...
  Accuracy: 58.79
  Validation took: 0:00:24

======== Epoch 3 / 5 ========
Training...
  Batch   311  of    312.    Loss: 1.37     Elapsed: 0:01:12.
  Average training loss: 1.79
  Training epoch took: 0:01:12
Running Validation...
  Accuracy: 58.79
  Validation took: 0:00:24

======== Epoch 4 / 5 ========
Training...
  Batch   311  of    312.    Loss: 1.37     Elapsed: 0:01:21.
  Average training loss: 1.79
  Training epoch took: 0:01:22
Running Validation...
  Accuracy: 58.79
  Validation took: 0:00:27

======== Epoch 5 / 5 ========
Training...
  Batch   311  of    312.

In [5]:
valLosses = [each.item() for each in list(zip(*CBouts))[0]]
print(valLosses)
print(trainingLosses)

NameError: name 'CBouts' is not defined

In [6]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
def plot_loss_vals(loss_vals, lv2, title="Training loss", start_idx=1):
    sns.set(style='darkgrid')
    sns.set(font_scale=1.5)
    plt.rcParams["figure.figsize"] = (12,6)
    plt.plot([i + start_idx for i in range(len(loss_vals[start_idx - 1:]))], loss_vals[start_idx - 1:], 'b-o', label="Training Loss (softmax)")
    plt.plot([i + start_idx for i in range(len(lv2[start_idx - 1:]))], lv2[start_idx - 1:], 'r-o', label="Val Loss (softmax)")
    plt.title(title)
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    plt.show()
plot_loss_vals(trainingLosses, valLosses, title="Training and Val Loss, FerDSSC, with 100 latent space", start_idx=2)

In [8]:
# evaluate test accuracy

class_correct = list(0. for i in range(len(id2idx)))
class_total = list(0. for i in range(len(id2idx)))
with torch.no_grad():
    for data in testLoader:
        inputs, labels = data["image"].to(device), data["label"].to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(labels.size(0)):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


KeyError: 0